# Download, Transcribe, Translate Videos

In [1]:
import os 
os.chdir('../')

## Download the video

In [2]:
url = "https://www.youtube.com/watch?v=iVaLCY9Zec0"

In [3]:
from gat.utils import *

video_filename = download_video(url)
# video_filename = "cache/95b_to5dmkY.webm"
filename_stem = "".join(video_filename.split('.')[:-1])

[youtube] Extracting URL: https://www.youtube.com/watch?v=iVaLCY9Zec0
[youtube] iVaLCY9Zec0: Downloading webpage
[youtube] iVaLCY9Zec0: Downloading web creator player API JSON
[youtube] iVaLCY9Zec0: Downloading mweb player API JSON
[info] iVaLCY9Zec0: Downloading 1 format(s): 303+251
[download] Destination: cache/iVaLCY9Zec0.f303.webm
[download] 100% of  417.86MiB in 00:00:43 at 9.54MiB/s      
[download] Destination: cache/iVaLCY9Zec0.f251.webm
[download] 100% of   23.87MiB in 00:00:02 at 8.73MiB/s     
[Merger] Merging formats into "cache/iVaLCY9Zec0.webm"
Deleting original file cache/iVaLCY9Zec0.f251.webm (pass -k to keep)
Deleting original file cache/iVaLCY9Zec0.f303.webm (pass -k to keep)


In [ ]:
from gat.transcription import get_whisper_prompt

whisper_prompt = get_whisper_prompt(video_filename)
whisper_prompt = "You are doing an Valorant analysis video. You might encounter words like: " + whisper_prompt

/home/zhangminhao/miniconda3/envs/video-translation/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
from gat.transcription import transcribe_whisper 

start, end, english = transcribe_whisper(video_filename, whisper_prompt=whisper_prompt)
save_srt(f"{filename_stem}.en.srt", start, end, english)

In [ ]:
start, end, english = load_srt(f"{filename_stem}.en.srt")

In [6]:
from gat.glossary_rag import GlossaryRAG
from gat.glossary_matcher import GlossaryMatcher

g_rag = GlossaryRAG("collection_name_1", "Snowflake/snowflake-arctic-embed-l-v2.0")
g_rag.load_from_dir("data")
g_mat = GlossaryMatcher()
g_mat.load_from_dir("data")

In [7]:
from gat.translator import DeepSeekTranslator 

translator = DeepSeekTranslator(g_rag, g_mat)

In [8]:
chinese = translator.translate_sentences(english)

Translating sentences: 100%|██████████| 156/156 [05:35<00:00,  2.15s/it]


In [9]:
save_srt(f"{filename_stem}.zh.srt", start, end, chinese)

save_bilingual_srt(f"{filename_stem}.en-zh.srt", start, end, english, chinese)